In [1]:
from htmd import *
import datetime

New HTMD version (1.0.22) is available. You are currently on (1.0.16). Use 'conda update htmd' to update to the new version.


First we load all simulations into a "simlist":

In [2]:
sims  = simlist(glob('/home/mauro/Documentos/MASTER/MSI/pub.htmd.org/CXCL12-confAnalysis/*/'), glob('/home/mauro/Documentos/MASTER/MSI/pub.htmd.org/CXCL12-confAnalysis/*/structure.pdb'))

Creating simlist: 100% (10/10) [###################################] eta 00:01 -


Then build a list of names ("traj" + number of the simulation) that will be used to assign each simulation to a variable.

In [3]:
name = []
for value in range(1,len(sims)+1):
    name.append("traj"+str(value))

In the following blocks the maximum distance for each simulation is computed and stored into a list. With this calculation what we want is to obtain the gratest maximum distance, in order to move the protein the same distance into que positive region of the 3D space. This approach avoids future problems while working with negative coordinates values.<br>
At the same time the program computes each maximum distance, a Molecule object is created, "wrapped" and "aligned". This commands fix the limits of the water box and the protein, so it will not move around the water box and will be aligned respect to the first simulation. Then, every simulation is prepared to enter in the following steps.

In [4]:
from htmd.molecule.util import maxDistance
max_dist=[]
sim_count = 0

In [6]:
for simul in sims:
    name[sim_count] = Molecule(simul)
    name[sim_count].wrap('protein')
    if sim_count == 0:
        name[sim_count].align('protein')
    else:
        name[sim_count].align('protein', refmol = name[0])
    W = maxDistance(name[sim_count], 'all')
    max_dist.append(W)
    sim_count +=1

In [7]:
W = int(np.round(max(max_dist)))
Ws= str(W)

Building a zeros matrix with a dimension equal to two times the max distance, plus 15, what we obtain is a space or 3D grid that will be used to store the counts for each position of the space.

In [8]:
my_matrix = np.zeros(((W*2)+15,(W*2)+15,(W*2)+15))

**Filling counts matrix**<br><br>
The next block takes each simulation, centers and moves it into the positive 3D space region, then a copy of the simulation is obtained and filtered to get just the oxigens of the water molecules. Then using this filtered atom selection, the program goes through every frame and every atom of the frame, getting the coordinates of the atoms (the coordinates are rounded in order to group the counts into voxels of 1x1x1 A) and adding +1 to the corresponding position of "my_matrix" space.<br><br>
In some simulations, some "strange" outlier atoms appear at extremely large coordinates values. To avoid taking into account this extreme atoms, each atom needs to be inside the space defined by the sphere with radius equal to the maximum distance previously obtained, and center equal to the position where every molecule is moved.

In [9]:
for simulation in name:
    simulation.center()
    simulation.moveBy([W,W,W])
    water = simulation.copy()
    water.filter('name OH2')
    for frame in range(0,len(water.coords[0][0])):
        for atom in range(0,len(water.coords)):
            coords_atom = water.coords[atom,:,frame]
            if (int(coords_atom[0])-W)**2+(int(coords_atom[1])-W)**2+(int(coords_atom[2])-W)**2 < W**2:
                my_matrix[int(np.round(water.coords[atom][0][frame]))][int(np.round(water.coords[atom][1][frame]))][int(np.round(water.coords[atom][2][frame]))] +=1

Boltzmann constant and tempterature that will be used to compute free energy

In [10]:
kb = 0.001987191
T = 298

**Gaussian occupancy**<br><br>
Using the following command implemented into the "scipy" package we apply a gaussian occupancy distribution over the neibouring voxels to the ones that contain at least one water oxygen. With this approach we get a more realistic isosurface representation, because it will take into account the different possible occupancies of the hydrogen atoms around the space occupied by the oxygen atom.

In [11]:
import scipy.ndimage

In [12]:
gaussian = scipy.ndimage.filters.gaussian_filter(my_matrix, 1.5)

Average matrix counts (divide by the number of different simulations included in the calculations):

In [13]:
next_matrix = gaussian/len(name)

***Free energy matrix calculation.***<br><br>
First the matrix is divided by the number of frames and atoms (oxigen atoms from water molecules) to obtain the probability matrix, that will have the information of the probability of having a water molecule in a certain position.<br>
Then a pseudocount is added to every position into the probability matrix, just to avoid "-inf" values when the natural logarithm be applied to the probability matrix. A very low value was chosen ("1e-40") to wider the difference between the isovalues of the regions where the water tends to be and the regions with lower water occupancy.<br>
Finally the natural logarithm is applied on the pseudocounted probability matrix, and multiplied by the temperature (25ºC = 298 K) and the Boltzmann constant.

In [14]:
last_matrix = -kb*T*np.log((next_matrix/(len(water.coords[0][0])*len(water.coords)))+1e-40)

**Writting ".cube" file**<br><br>
As the documentation specifies, for the "writeVoxels" function, the minimum and maximum coordinates of the 3D space must be provided in a "np.array" format, and the step for each coordinate.

In [15]:
my_min = np.array([0,0,0])
my_max = np.array([(W*2)+15,(W*2)+15,(W*2)+15])
steps = np.array([1,1,1])

In [16]:
htmd.molecule.util.writeVoxels(last_matrix,"my_Voxel_average.cube",my_min,my_max,steps)

**Visualization** <br><br>
Finally, selecting one of the stored simulations and loading into them the ".cube" file, it can be seen the final result. <br>
The best isovalue is around 7.4-7.5

In [18]:
name[1].view()